In [1]:
import re, os, sys, unicodedata
import pandas as pd
import numpy as np

In [2]:
import optparse
import dedupe
from unidecode import unidecode

#### Read Data - Filed Description:
recId: entites with the same recId refer to the same entity. this is the ground truth, join to df0 by this column to get the 'answers'. Drop this field for now --> save in answers df 
givenname:   
surname:  
post code:  
suburb:

In [3]:
fp0 = '/Users/gizelleguerra/Documents/fedex_cip/NCvoterdata/nc0.csv'
fp1 = '/Users/gizelleguerra/Documents/fedex_cip/NCvoterdata/nc1.csv'

In [4]:
# ground truth - no duplicates in this file
df0 = pd.read_csv(fp0, dtype={'recid':np.int64, 'givenname':str, 'surname':str, 'suburb':str, 'postcode':str})

In [5]:
# messy data to be cleaned and matched to data in df0
df1 = pd.read_csv(fp1, dtype={'recid':np.int64, 'givenname':str, 'surname':str, 'suburb':str, 'postcode':str})

In [6]:
df0

,recid,givenname,surname,suburb,postcode
0,7852009,kadelyn,gragnani,waxhaw,28|73
1,3138435,ronel,carter,washington,2788g
2,6117295,jason,roehrig,millers ceeek,286s1
3,6401695,antrea,mure,greensboro,27410
4,5202204,caedar,oliver,jacksonville,28542
...,...,...,...,...,...
999995,6614741,virginia,angle,charlotte,28205
999996,2840680,james,stone,eden,27288
999997,5781566,rebecca,allen,willow springs,27592
999998,1737677,julie,cummings,fremont,27830


In [7]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   recid      1000000 non-null  int64 
 1   givenname  999928 non-null   object
 2   surname    999980 non-null   object
 3   suburb     999953 non-null   object
 4   postcode   999942 non-null   object
dtypes: int64(1), object(4)
memory usage: 38.1+ MB


In [7]:
df1

,recid,givenname,surname,suburb,postcode
0,4194454,ashlev,batts,raleigh,276o6
1,4263711,tiffamy,perez,vass,28354
2,8089896,david,murry,charlogte,28268
3,7879554,shamkea,hutchinson,rocky mount,278o4
4,4338459,john,snyddr,louisburg,27s49
...,...,...,...,...,...
999995,3190016,clyde,wilder,raleigh,27614
999996,1890704,horace,swain,wilmington,28405
999997,6483474,peggy,honeycutt,angier,27501
999998,412753,ethel,hamm,elm city,27822


In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   recid      1000000 non-null  int64 
 1   givenname  999942 non-null   object
 2   surname    999983 non-null   object
 3   suburb     999951 non-null   object
 4   postcode   999943 non-null   object
dtypes: int64(1), object(4)
memory usage: 38.1+ MB


In [8]:
df_comb = pd.concat([df0, df1])
df_comb

,recid,givenname,surname,suburb,postcode
0,7852009,kadelyn,gragnani,waxhaw,28|73
1,3138435,ronel,carter,washington,2788g
2,6117295,jason,roehrig,millers ceeek,286s1
3,6401695,antrea,mure,greensboro,27410
4,5202204,caedar,oliver,jacksonville,28542
...,...,...,...,...,...
999995,3190016,clyde,wilder,raleigh,27614
999996,1890704,horace,swain,wilmington,28405
999997,6483474,peggy,honeycutt,angier,27501
999998,412753,ethel,hamm,elm city,27822


In [9]:
answers = pd.DataFrame(columns=['recid_0', 'recid_1'])
answers['recid_0'] = df0['recid']
answers['recid_1'] = df1['recid']
#answers.drop(answers.tail(1).index,inplace=True)
answers

,recid_0,recid_1
0,7852009,4194454
1,3138435,4263711
2,6117295,8089896
3,6401695,7879554
4,5202204,4338459
...,...,...
999995,6614741,3190016
999996,2840680,1890704
999997,5781566,6483474
999998,1737677,412753


In [12]:
answers2 = df_comb['recid']

In [13]:
df1.drop(columns = ['recid'], inplace = True)
df0.drop(columns = ['recid'], inplace = True)
df_comb.drop(columns = ['recid'], inplace = True)

#### Clean Data

In [16]:
## check for nulls in all fields and drop them - dont do this yet though, will mess up answer index
#df1.dropna(subset=['givenname', 'surname'], inplace = True)
#df0.dropna(subset=['givenname', 'surname'], inplace = Ture)
df_comb.dropna(how = 'all', inplace = True)

In [18]:
col_list = df1.columns
col_list[:-1]

Index(['givenname', 'surname', 'suburb'], dtype='object')

In [ ]:
#### clean up individual dfs

In [19]:
# process string fields (not zip code) and remove numbers and special characters
# str() conversion seems superfluous here (is also slow!) but there is some weird data that throws errors in the dataset w/o it 
for col in col_list[:-1]:
    df1.loc[:,col] = df1[col].apply(lambda x: str(x)).str.replace('[^A-Za-z]+', '', regex=True)

In [20]:
# process zip code field and remove letters and special characters
df1.loc[:,'postcode'] = df1['postcode'].apply(lambda x: str(x)).str.replace('o', '0', regex=True).replace('[^0-9]+', '', regex=True)
df1.head()

,givenname,surname,suburb,postcode
0,ashlev,batts,raleigh,27606
1,tiffamy,perez,vass,28354
2,david,murry,charlogte,28268
3,shamkea,hutchinson,rockymount,27804
4,john,snyddr,louisburg,2749


In [21]:
# process string fields (not zip code) and remove numbers and special characters
# str() conversion seems superfluous here (is also slow!) but there is some weird data that throws errors in the dataset w/o it 
for col in col_list[:-1]:
    df0.loc[:,col] = df0[col].apply(lambda x: str(x)).str.replace('[^A-Za-z]+', '', regex=True)

In [22]:
# process zip code field and remove letters and special characters
df0.loc[:,'postcode'] = df0['postcode'].apply(lambda x: str(x)).str.replace('o', '0', regex=True).replace('[^0-9]+', '', regex=True)
df0.head()

,givenname,surname,suburb,postcode
0,kadelyn,gragnani,waxhaw,2873
1,ronel,carter,washington,2788
2,jason,roehrig,millersceeek,2861
3,antrea,mure,greensboro,27410
4,caedar,oliver,jacksonville,28542


In [ ]:
# Save files as .csvs
df0.to_csv('nc0_clean.zip', index=False)
df1.to_csv('nc1_clean.zip', index=False)

In [ ]:
#### clean up combined file

In [23]:
# process string fields (not zip code) and remove numbers and special characters
# str() conversion seems superfluous here (is also slow!) but there is some weird data that throws errors in the dataset w/o it 
for col in col_list[:-1]:
    df_comb.loc[:,col] = df_comb[col].apply(lambda x: str(x)).str.replace('[^A-Za-z]+', '', regex=True)

In [24]:
# process zip code field and remove letters and special characters
df_comb.loc[:,'postcode'] = df_comb['postcode'].apply(lambda x: str(x)).str.replace('o', '0', regex=True).replace('[^0-9]+', '', regex=True)
df_comb.head()

,givenname,surname,suburb,postcode
0,kadelyn,gragnani,waxhaw,2873
1,ronel,carter,washington,2788
2,jason,roehrig,millersceeek,2861
3,antrea,mure,greensboro,27410
4,caedar,oliver,jacksonville,28542


In [25]:
# save combined df as .csv
df_comb.to_csv('combined_clean.zip', index=False)